In [9]:
#load data 

import pandas as pd
import pickle
import numpy as np
from sklearn.svm import SVR
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV

X_total = pickle.load(open('X', 'rb'))
Y_fare_total = pickle.load(open('Y_fare', 'rb'))
Y_tip_total = pickle.load(open('Y_tip', 'rb'))

In [10]:
traingRecordNumber = 5000
X = X_total.as_matrix()[0:traingRecordNumber,:]


y = Y_fare_total.tolist()[0:traingRecordNumber]

# Training

In [ ]:
model = SVR()
#scores4 = cross_val_score(model, X2, y, cv=4,scoring='neg_mean_squared_error',n_jobs=-1)

param_grid = [
 {'C': [1, 10, 100, 1000], 'gamma': [0.01,0.001, 0.0001], 'kernel': ['linear']},
 {'C': [1, 10, 100, 1000], 'gamma': [0.01,0.001, 0.0001], 'kernel': ['rbf']},
 {'C': [1, 10, 100, 1000], 'gamma': [0.01,0.001, 0.0001], 'degree':[3,5,7,9] ,'kernel': ['poly']}
 # {'hidden_layer_sizes': [(20,20),(10,10),(20,30)]},
 ]

clf = GridSearchCV(model, param_grid, cv=4,scoring='r2',
                       n_jobs=-1)

clf.fit(X,y)

clf.cv_results_

In [13]:
clf.best_score_

0.34656446763828785

In [15]:
clf.best_params_

21

In [ ]:
scores2 = cross_val_score(svr_rbf, X2, y, cv=4,scoring='neg_mean_squared_error',n_jobs=-1)

In [ ]:
scores2

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor()
scores3 = cross_val_score(model, X2, y, cv=4,scoring='neg_mean_squared_error',n_jobs=-1)

In [ ]:
scores3

In [ ]:
from sklearn import linear_model
model = linear_model.SGDRegressor()
scores3 = cross_val_score(model, X2, y, cv=4,scoring='neg_mean_squared_error',n_jobs=-1)

In [ ]:
scores3

In [ ]:
from sklearn.neural_network import MLPRegressor

model = MLPRegressor(hidden_layer_sizes=(100,30))
scores4 = cross_val_score(model, X2, y, cv=4,scoring='neg_mean_squared_error',n_jobs=-1)

In [ ]:
scores4